In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.getOrCreate()

In [9]:
"""If you don’t want to specify the schema, Spark can infer schema
from a sample at a lesser cost. For example, you can use the
samplingRatio option:
"""

sampleDF  = spark.read.csv("sf-fire-calls.csv"
                          ,samplingRatio=0.001
                          , header=True)

In [10]:
sampleDF.printSchema()

root
 |-- CallNumber: string (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: string (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: string (nullable = true)
 |-- ALSUnit: string (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: string (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: string (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |-- Sup

In [11]:
fire_df = spark.read.csv('sf-fire-calls.csv', header=True, schema=sampleDF.schema)

In [12]:
fire_df.printSchema()

root
 |-- CallNumber: string (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: string (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: string (nullable = true)
 |-- ALSUnit: string (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: string (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: string (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |-- Sup

Now that you have a distributed DataFrame composed of San Francisco Fire Department calls in memory, the first thing you as a developer will want to do is examine
your data to see what the columns look like. Are they of the correct types? Do any of
them need to be converted to different types? Do they have null values?

### Projections and filters:
A <i>projection</i> in relational parlance is a way to return only the
rows matching a certain relational condition by using filters. In Spark, projections are
done with the <b>select()</b> method, while filters can be expressed using the <b>filter()</b> or
<b>where()</b> method. We can use this technique to examine specific aspects of our SF Fire
Department data set:

In [13]:
import pyspark.sql.functions as fn

In [14]:
from pyspark.sql.functions import *

In [15]:
few_fire_df = (fire_df
              .select("IncidentNumber", "AvailableDtTm", "CallType")
              .where(col("CallType") != "Medical Incident"))

few_fire_df.show(5,truncate=False)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



In [17]:
ew_fire_df = (fire_df
              .select("IncidentNumber", "AvailableDtTm", "CallType")
              .where("CallType != 'Medical Incident'"))

few_fire_df.show(5,truncate=False)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



In [18]:
(fire_df
 .select("IncidentNumber", "AvailableDtTm", "CallType")
 .where(fire_df["CallType"] != "Medical Incident")).show(5, truncate=False)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



In [19]:
fire_df.select('CallType')\
.where(col('CallType').isNotNull())\
.show(10,truncate=False)

+----------------+
|CallType        |
+----------------+
|Structure Fire  |
|Medical Incident|
|Medical Incident|
|Vehicle Fire    |
|Alarms          |
|Structure Fire  |
|Alarms          |
|Alarms          |
|Medical Incident|
|Medical Incident|
+----------------+
only showing top 10 rows



We can list the distinct call types in the data set using these queries:

In [37]:
fire_df.select('CallType')\
.where(col('CallType').isNotNull())\
.distinct()\
.show(20,truncate=False)

+--------------------------------------------+
|CallType                                    |
+--------------------------------------------+
|Elevator / Escalator Rescue                 |
|Marine Fire                                 |
|Aircraft Emergency                          |
|Confined Space / Structure Collapse         |
|Administrative                              |
|Alarms                                      |
|Odor (Strange / Unknown)                    |
|Citizen Assist / Service Call               |
|HazMat                                      |
|Watercraft in Distress                      |
|Explosion                                   |
|Oil Spill                                   |
|Vehicle Fire                                |
|Suspicious Package                          |
|Extrication / Entrapped (Machinery, Vehicle)|
|Other                                       |
|Outside Fire                                |
|Traffic Collision                           |
|Assist Polic

In [36]:
fire_df.select('CallType')\
.where(col('CallType').isNotNull())\
.distinct()\
.sort('CallType')\
.show(30,truncate=False)

+--------------------------------------------+
|CallType                                    |
+--------------------------------------------+
|Administrative                              |
|Aircraft Emergency                          |
|Alarms                                      |
|Assist Police                               |
|Citizen Assist / Service Call               |
|Confined Space / Structure Collapse         |
|Electrical Hazard                           |
|Elevator / Escalator Rescue                 |
|Explosion                                   |
|Extrication / Entrapped (Machinery, Vehicle)|
|Fuel Spill                                  |
|Gas Leak (Natural and LP Gases)             |
|HazMat                                      |
|High Angle Rescue                           |
|Industrial Accidents                        |
|Marine Fire                                 |
|Medical Incident                            |
|Mutual Aid / Assist Outside Agency          |
|Odor (Strang

What if we want to know how many distinct <b>CallTypes</b> were recorded as the causes
of the fire calls? These simple and expressive queries do the job:

In [33]:
fire_df.select('CallType')\
.where(col('CallType').isNotNull())\
.distinct()\
.count()

30

In [38]:
fire_df.select('CallType')\
.where(col('CallType').isNotNull())\
.agg(fn.countDistinct('CallType').alias('DistinctCallTypes')).show()

+-----------------+
|DistinctCallTypes|
+-----------------+
|               30|
+-----------------+



### Renaming, adding, and dropping columns:

Rename columns with the <b>withColumnRenamed()</b>
method. For instance, let’s change the name of our <b>Delay</b> column to <b>ResponseDelayedinMins</b> and take a look at the response times that were longer than five minutes:

In [39]:
new_fire_df = fire_df.withColumnRenamed('Delay',"ResponseDelayedMins")

In [40]:
new_fire_df.printSchema()

root
 |-- CallNumber: string (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: string (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: string (nullable = true)
 |-- ALSUnit: string (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: string (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: string (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |-- Sup

<b><font color='red'>Note:</font></b> Because DataFrame transformations are immutable, when we
rename a column using <b>withColumnRenamed()</b> we get a new Data‐
Frame while retaining the original with the old column name.

In [41]:
fire_df.printSchema()

root
 |-- CallNumber: string (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: string (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: string (nullable = true)
 |-- ALSUnit: string (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: string (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: string (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |-- Sup

In [42]:
(new_fire_df.select('ResponseDelayedMins').where(col('ResponseDelayedMins')>5)
.show(5,False))

+-------------------+
|ResponseDelayedMins|
+-------------------+
|6.25               |
|7.25               |
|11.916667          |
|8.633333           |
|95.28333           |
+-------------------+
only showing top 5 rows



In [44]:
(new_fire_df.select('ResponseDelayedMins').where('ResponseDelayedMins>5')
.show(5,False))

+-------------------+
|ResponseDelayedMins|
+-------------------+
|6.25               |
|7.25               |
|11.916667          |
|8.633333           |
|95.28333           |
+-------------------+
only showing top 5 rows



<b>--</b> Modifying the contents of a column or its type are common operations during data exploration. In some cases the data is raw or dirty, or its types are not amenable to being supplied as arguments to relational operators.<br>
<b>--</b> For example, in our <b>SF Fire</b>
Department data set, the columns <b>CallDate , WatchDate , and AlarmDtTm</b> are strings
rather than either <b>Unix timestamps</b> or <b>SQL dates</b>, both of which Spark supports and
can easily manipulate during transformations or actions (e.g., during a date- or time-
based analysis of the data).

### Steps:
<ol>
    <li>Convert the existing column’s data type from string to a Spark-supported timestamp.</li>
    <li>Use the new format specified in the format string <b>"MM/dd/yyyy"</b> or <b>"MM/dd/yyyy hh:mm:ss a"</b> where appropriate.</li>
    <li>After converting to the new data type, drop() the old column and append the new one specified in the first argument to the <b>withColumn()</b> method.</li>
    <li>Assign the new modified DataFrame to <b>fire_ts_df</b></li>
</ol>

In [46]:
fire_ts_df = new_fire_df.withColumn('IncidentDate',fn.to_timestamp(col('CallDate'),'MM/dd/yyyy'))\
                        .drop('CallDate')\
                        .withColumn('OnWatchDate',to_timestamp(col('WatchDate'),'MM/dd/yyyy'))\
                        .drop('WatchDate')\
                        .withColumn('AvailableDtTs',to_timestamp(col('AvailableDtTm'),'MM/dd/yyyy hh:mm:ss a'))\
                        .drop('AvailableDtTm')

In [47]:
fire_ts_df.printSchema()

root
 |-- CallNumber: string (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: string (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: string (nullable = true)
 |-- ALSUnit: string (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: string (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: string (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |-- SupervisorDistrict: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Location: string (nullable = true)


In [49]:
ire_ts_df_2 = new_fire_df.withColumn('CallDate',fn.to_timestamp(col('CallDate'),'MM/dd/yyyy'))\
                        .withColumn('WatchDate',to_timestamp(col('WatchDate'),'MM/dd/yyyy'))\
                        .withColumn('AvailableDtTm',to_timestamp(col('AvailableDtTm'),'MM/dd/yyyy hh:mm:ss a'))

In [50]:
new_fire_df.printSchema()

root
 |-- CallNumber: string (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: string (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: string (nullable = true)
 |-- ALSUnit: string (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: string (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: string (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |-- Sup

In [51]:
ire_ts_df_2.printSchema()

root
 |-- CallNumber: string (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: string (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: timestamp (nullable = true)
 |-- WatchDate: timestamp (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: timestamp (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: string (nullable = true)
 |-- ALSUnit: string (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: string (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: string (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)

In [52]:
fire_ts_df.select(['IncidentDate','OnWatchDate','AvailableDtTs']).show(5,False)

+-------------------+-------------------+-------------------+
|IncidentDate       |OnWatchDate        |AvailableDtTs      |
+-------------------+-------------------+-------------------+
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:51:44|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 03:01:18|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 02:39:50|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 04:16:46|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 06:01:58|
+-------------------+-------------------+-------------------+
only showing top 5 rows



Now that we have modified the dates, we can query using functions from <b>spark.sql.functions</b> like <b>month() , year() , and day()</b> to explore our data further.

In [53]:
fire_ts_df.select(fn.year('IncidentDate')).show()

+------------------+
|year(IncidentDate)|
+------------------+
|              2002|
|              2002|
|              2002|
|              2002|
|              2002|
|              2002|
|              2002|
|              2002|
|              2002|
|              2002|
|              2002|
|              2002|
|              2002|
|              2002|
|              2002|
|              2002|
|              2002|
|              2002|
|              2002|
|              2002|
+------------------+
only showing top 20 rows



In [54]:
fire_ts_df.select(fn.year('IncidentDate'))\
.distinct()\
.show()

+------------------+
|year(IncidentDate)|
+------------------+
|              2003|
|              2018|
|              2015|
|              2006|
|              2013|
|              2014|
|              2012|
|              2009|
|              2016|
|              2005|
|              2010|
|              2011|
|              2008|
|              2017|
|              2002|
|              2007|
|              2004|
|              2001|
|              2000|
+------------------+



In [56]:
fire_ts_df.select(fn.year('IncidentDate'))\
.distinct()\
.orderBy(fn.year('IncidentDate'))\
.show()

+------------------+
|year(IncidentDate)|
+------------------+
|              2000|
|              2001|
|              2002|
|              2003|
|              2004|
|              2005|
|              2006|
|              2007|
|              2008|
|              2009|
|              2010|
|              2011|
|              2012|
|              2013|
|              2014|
|              2015|
|              2016|
|              2017|
|              2018|
+------------------+



In [58]:
fire_ts_df.select(fn.year('IncidentDate').alias('IncidentYear'))\
.distinct()\
.orderBy('IncidentYear')\
.show()

+------------+
|IncidentYear|
+------------+
|        2000|
|        2001|
|        2002|
|        2003|
|        2004|
|        2005|
|        2006|
|        2007|
|        2008|
|        2009|
|        2010|
|        2011|
|        2012|
|        2013|
|        2014|
|        2015|
|        2016|
|        2017|
|        2018|
+------------+



So far, we have explored a number of common data operations: reading and writing DataFrames; defining a schema and using it when reading in a DataFrame; projecting and filtering selected columns from an existing DataFrame; and modifying, renaming, and dropping columns.

## Aggregations.

### What were the most common types of fire calls? 

In [59]:
(fire_ts_df.select('CallType')
 .where(col('CallType').isNotNull())
 .groupBy('CallType')
 .count()
 .orderBy('count',ascending=False)
 .show(10,False)
)

+-------------------------------+------+
|CallType                       |count |
+-------------------------------+------+
|Medical Incident               |113794|
|Structure Fire                 |23319 |
|Alarms                         |19406 |
|Traffic Collision              |7013  |
|Citizen Assist / Service Call  |2524  |
|Other                          |2166  |
|Outside Fire                   |2094  |
|Vehicle Fire                   |854   |
|Gas Leak (Natural and LP Gases)|764   |
|Water Rescue                   |755   |
+-------------------------------+------+
only showing top 10 rows



In [60]:
(fire_ts_df.select('CallType')
 .where(col('CallType').isNotNull())
 .groupBy('CallType')
 .agg(count(col('CallType')).alias('CallTypeCount'))
 .orderBy('CallTypeCount',ascending=False)
 .show(10,False)
)

+-------------------------------+-------------+
|CallType                       |CallTypeCount|
+-------------------------------+-------------+
|Medical Incident               |113794       |
|Structure Fire                 |23319        |
|Alarms                         |19406        |
|Traffic Collision              |7013         |
|Citizen Assist / Service Call  |2524         |
|Other                          |2166         |
|Outside Fire                   |2094         |
|Vehicle Fire                   |854          |
|Gas Leak (Natural and LP Gases)|764          |
|Water Rescue                   |755          |
+-------------------------------+-------------+
only showing top 10 rows



### What zip codes accounted for the most calls?

In [61]:
(fire_ts_df.select('zipCode')
 .where(col('zipCode').isNotNull())
 .groupBy('zipCode')
 .count()
 .orderBy('count',ascending=False)
 .show(10,False)
)

+-------+-----+
|zipCode|count|
+-------+-----+
|94102  |21840|
|94103  |20897|
|94110  |14801|
|94109  |14686|
|94124  |9236 |
|94112  |8421 |
|94115  |7812 |
|94107  |6941 |
|94122  |6355 |
|94133  |6246 |
+-------+-----+
only showing top 10 rows



## Other common DataFrame operations.
DataFrame API provides descriptive statistical methods like <b>min() , max() , sum() , and avg()</b>.

#### compute the sum of alarms, the average response time, and the minimum and maximum response times to all fire calls in our SF Fire Department data set

In [63]:
(fire_ts_df
 .select(fn.sum("NumAlarms"),fn.avg("ResponseDelayedMins")
         ,fn.min("ResponseDelayedMins"),fn.max("ResponseDelayedMins"))
 .show()
)

+--------------+------------------------+------------------------+------------------------+
|sum(NumAlarms)|avg(ResponseDelayedMins)|min(ResponseDelayedMins)|max(ResponseDelayedMins)|
+--------------+------------------------+------------------------+------------------------+
|      176170.0|      3.8923641541750342|             0.016666668|                    99.9|
+--------------+------------------------+------------------------+------------------------+



In [65]:
(fire_ts_df
 .select(fn.sum("NumAlarms").alias("Alarms SUM "),fn.avg("ResponseDelayedMins").alias("Avg Response Mins")
         ,fn.min("ResponseDelayedMins").alias("Min Response Mins")
         ,fn.max("ResponseDelayedMins").alias("Max Response Mins"))
 .show()
)

+-----------+------------------+-----------------+-----------------+
|Alarms SUM | Avg Response Mins|Min Response Mins|Max Response Mins|
+-----------+------------------+-----------------+-----------------+
|   176170.0|3.8923641541750342|      0.016666668|             99.9|
+-----------+------------------+-----------------+-----------------+



For more advanced statistical needs common with data science workloads, read the
API documentation for methods like <b>stat() , describe() , correlation() ,
covariance() , sampleBy() , approxQuantile() , frequentItems()</b> , and so on.